In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read file: USD_CAD_M15_2005_202206
FILE_NAME = 'USD_CAD_M15_2005_202206'
#change directroy
file = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\{FILE_NAME}.csv', encoding='utf-8')

In [3]:
#check for any missing value
file.isnull().sum()

datetime    0
open        0
high        0
low         0
close       0
dtype: int64

In [4]:
data = file.copy()
#use lastest 250 row of data for visual and pattern detect
data = data.tail(250)
data

datetime     open     high      low    close
439877  2022-06-28T09:15:00.000000000Z  1.28224  1.28266  1.28222  1.28248
439878  2022-06-28T09:30:00.000000000Z  1.28248  1.28343  1.28244  1.28341
439879  2022-06-28T09:45:00.000000000Z  1.28341  1.28368  1.28318  1.28344
439880  2022-06-28T10:00:00.000000000Z  1.28348  1.28371  1.28330  1.28342
439881  2022-06-28T10:15:00.000000000Z  1.28342  1.28361  1.28308  1.28324
...                                ...      ...      ...      ...      ...
440122  2022-06-30T22:30:00.000000000Z  1.28832  1.28832  1.28792  1.28792
440123  2022-06-30T22:45:00.000000000Z  1.28792  1.28808  1.28724  1.28732
440124  2022-06-30T23:00:00.000000000Z  1.28732  1.28778  1.28718  1.28772
440125  2022-06-30T23:15:00.000000000Z  1.28775  1.28800  1.28758  1.28762
440126  2022-06-30T23:30:00.000000000Z  1.28763  1.28786  1.28760  1.28774

[250 rows x 5 columns]

In [5]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

fig.show()


In [6]:
def check_ascending_order(nums):
    count = 0
    if len(nums) != 1:
        for i in range(len(nums)):
            if nums[i] >= nums[0]:
                count+=1
                
    if count == len(nums):
        return True
    else:
        return False
    
def check_descending_order(nums):
    count = 0
    if len(nums) != 1:
        for i in range(len(nums)):
            if nums[i] <= nums[0]:
                count+=1
                
    if count == len(nums):
        return True
    else:
        return False       
    
def list_handle(nested_list):
    
    result = []
    for ls in nested_list:
        if len(ls) > 0:
            new_ls = [ls[0], ls[len(ls)-1]]
            result.append(new_ls)
        else:
            continue
    return result

def converge(high, low):
    #trading volume decrease
    if (high[-1] - low[-1]) < high[0] - low[0]:
        return True
    else:
        return False

def flag(dataframe, datetime_ls, column_name): 
    dataframe[column_name] = 0
    temp = []
    for ls in datetime_ls:
        for num in ls:
            temp.append(num)
    dataframe = dataframe.reset_index()
    for i in temp:
        for j in range(len(dataframe['datetime'])):
            if dataframe['datetime'][j] == i:
                dataframe[column_name][j] =1
    return dataframe

# def diverge(high, low):
#      #trading volume increase
#     if (high[-1] - low[-1])> high[0] - low[0]:
#         return True
#     else:
#         return False

def rising_wedge(high,low, datetime,length = 5):
    #length: minimum number of candlestick bar that satisfy the conditions
    high_ls = []
    high_datetime_ls = []
    high_bottom_line_ls = []
    
    for i in range(len(high)):
    # rising wedge have to satisfy 3 conditions:
    # high and low both are rising
    # converge (trading volumne is decreasing)
        if check_ascending_order(high[i:i+length]) and check_ascending_order(low[i:i+length]) and converge(high[i:i+length],low[i:i+length]):
    
            high_ls.append(high[i:i+length])
            high_bottom_line_ls.append(low[i:i+length])
            high_datetime_ls.append(datetime[i:i+length])
    
    return high_ls, high_datetime_ls, high_bottom_line_ls
        

def falling_wedge(high,low, datetime,length = 5):
    #length: minimum number of candlestick bar that satisfy the conditions
    low_ls = []
    low_datetime_ls = []
    low_bottom_line_ls = []

    for i in range(len(high)):
    # falling wedge have to satisfy 3 conditions:
    # high and low both are falling
    # converge (trading volumne is decreasing)
        if  check_descending_order(high[i:i+length]) and check_descending_order(low[i:i+length]) and converge(high[i:i+length],low[i:i+length]):
            low_ls.append(low[i:i+length])
            low_bottom_line_ls.append(high[i:i+length])
            low_datetime_ls.append(datetime[i:i+length])
    
    return low_ls, low_datetime_ls, low_bottom_line_ls

In [15]:
high = list(data['high'])
low = list(data['low'])
datetime =  list(data['datetime'])
high_ls, high_datetime_ls, high_bottom_line_ls = rising_wedge(high,low,datetime,length =10)
#number of pattern found
print(str(len(high_ls)) + ' rising wedge pattern found')


15 rising wedge pattern found


In [16]:
# number of pattern found
high = list(data['high'])
low = list(data['low'])
datetime =  list(data['datetime'])
low_ls, low_datetime_ls, low_bottom_line_ls = falling_wedge(high,low,datetime,length = 10)
print(str(len(low_ls)) + ' falling wedge pattern found')

18 falling wedge pattern found


In [17]:
# mark the dataframe falling wedge = 1, otherwise 0
test_data = flag(data, low_datetime_ls,'Falling Wedge')

# collect results into a list for visualization
final_a, final_b, final_c = [],[],[]
for i in range(len(test_data['Falling Wedge'])-1):
    if test_data['Falling Wedge'][i] == 1:
        final_a.append(test_data['low'][i])
        final_b.append(test_data['high'][i])
        final_c.append(test_data['datetime'][i])
        if test_data['Falling Wedge'][i+1] == 0:
            final_a.append('X')
            final_b.append('X')
            final_c.append('X')
temp =[]
result_low, result_low_bottom, result_datetime = [],[],[]
for i in final_a:
    temp.append(i)
    if i =='X':
        temp.pop()
        result_low.append(temp)
        temp = []
temp =[]
for i in final_b:
    temp.append(i)
    if i =='X':
        temp.pop()
        result_low_bottom.append(temp)
        temp = []
temp =[]
for i in final_c:
    temp.append(i)
    if i =='X':
        temp.pop()
        result_datetime.append(temp)
        temp = []

# connect the first and last number in the list for visualization
result_low = list_handle(result_low)
result_datetime = list_handle(result_datetime)
result_low_bottom = list_handle(result_low_bottom)

fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

for i in range(len(result_low)):
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_low[i] , mode='lines', marker_color = 'red', showlegend=False, name = 'Falling Wedge'))
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_low_bottom[i], mode='lines', marker_color = 'red',showlegend=False))

fig.update_layout(title="Falling Wedge")
fig.show()

In [16]:
# mark the dataframe Rising wedge = 1, otherwise 0
test_data = flag(data, high_datetime_ls,'Rising Wedge')

# collect results into a list for visualization
final_a, final_b, final_c = [],[],[]
for i in range(len(test_data['Rising Wedge'])-1):
    if test_data['Rising Wedge'][i] == 1:
        final_a.append(test_data['low'][i])
        final_b.append(test_data['high'][i])
        final_c.append(test_data['datetime'][i])
        if test_data['Rising Wedge'][i+1] == 0:
            # print('r')
            final_a.append('X')
            final_b.append('X')
            final_c.append('X')

temp =[]
result_high, result_high_bottom, result_datetime = [],[],[]

for i in final_a:
    temp.append(i)
    if i =='X':
        temp.pop()
        result_high.append(temp)
        temp = []
temp =[]
for i in final_b:
    temp.append(i)
    if i =='X':
        temp.pop()
        result_high_bottom.append(temp)
        temp = []
temp =[]
for i in final_c:
    temp.append(i)
    if i =='X':
        temp.pop()
        result_datetime.append(temp)
        temp = []
        
# connect the first and last number in the list for visualization
result_high = list_handle(result_high)
result_datetime = list_handle(result_datetime)
result_high_bottom = list_handle(result_high_bottom)

fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

for i in range(len(result_high)):
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_high[i] , mode='lines', marker_color = 'green', showlegend=False, name = 'Rising Wedge'))
    fig.add_trace(go.Scatter(x=result_datetime[i], y=result_high_bottom[i], mode='lines', marker_color = 'green',showlegend=False))

fig.update_layout(title="Rising Wedge")
fig.show()

In [17]:
# dataframe with flagged column rising/falling wedge
test_data

index                        datetime     open     high      low  \
0    439927  2022-06-28T21:45:00.000000000Z  1.28724  1.28730  1.28718   
1    439928  2022-06-28T22:00:00.000000000Z  1.28710  1.28738  1.28704   
2    439929  2022-06-28T22:15:00.000000000Z  1.28738  1.28759  1.28726   
3    439930  2022-06-28T22:30:00.000000000Z  1.28748  1.28765  1.28734   
4    439931  2022-06-28T22:45:00.000000000Z  1.28738  1.28746  1.28713   
..      ...                             ...      ...      ...      ...   
195  440122  2022-06-30T22:30:00.000000000Z  1.28832  1.28832  1.28792   
196  440123  2022-06-30T22:45:00.000000000Z  1.28792  1.28808  1.28724   
197  440124  2022-06-30T23:00:00.000000000Z  1.28732  1.28778  1.28718   
198  440125  2022-06-30T23:15:00.000000000Z  1.28775  1.28800  1.28758   
199  440126  2022-06-30T23:30:00.000000000Z  1.28763  1.28786  1.28760   

       close  Rising Wedge  Falling Wedge  
0    1.28718             0              0  
1    1.28734             0              0  
2    1.28748             0              0  
3    1.28735             0              0  
4    1.28716             0              0  
..       ...           ...            ...  
195  1.28792             0              0  
196  1.28732             0              0  
197  1.28772             1              0  
198  1.28762             1              0  
199  1.28774             1              0  

[200 rows x 8 columns]